In [31]:
import csv
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
import pandas as pd 
from time import sleep

In [32]:
# initial variables
url = 'https://twitter.com/i/flow/login'
user = 'username'
password = 'password'
search_term = 'floyd mayweather'
col_name = ['Username', 'Id', 'PostDate', 'Text', 'Reply', 'Retweet', 'Like', 'TweetURL']
post_count = 50
reply_count = '0'
retweet_count = '0'
like_count = '0'
records = []

In [33]:
# login to twitter
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(url)
driver.implicitly_wait(10)

user_tag = driver.find_element_by_xpath('//input[@name="text"]')
user_tag.send_keys(user)
user_tag.send_keys(Keys.RETURN)
driver.implicitly_wait(3)

pass_tag = driver.find_element_by_xpath('//input[@name="password"]')
pass_tag.send_keys(password)
pass_tag.send_keys(Keys.RETURN)
driver.implicitly_wait(3)

# searching
search_input = driver.find_element_by_xpath('//input[@aria-label="Search query"]')
search_input.send_keys(search_term)
search_input.send_keys(Keys.RETURN)
driver.implicitly_wait(5)

# navigate to historical 'latest' tab
#driver.find_element_by_link_text('Latest').click()

print('logged in')

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [C:\Users\Pleum\.wdm\drivers\chromedriver\win32\102.0.5005.61\chromedriver.exe] found in cache


logged in


In [34]:
y = driver.execute_script("return window.pageYOffset;")
# scroll down
while True:
    # pull all tweet
    divs = driver.find_elements_by_xpath('//div[@data-testid="cellInnerDiv"]')
    for div in divs:
        try:
            tw_username = div.find_element_by_xpath('.//span').text
            tw_id = div.find_element_by_xpath('.//div/div/div/div[2]/div[2]/div[1]/div/div/div[1]/div[1]/div/div[2]/div/a/div/div/span').text
            tw_date = div.find_element_by_xpath('.//time').get_attribute('datetime')
            tw_text = div.find_element_by_xpath('.//div[@data-testid="tweetText"]').text
            tw_url = 'https://twitter.com/' + tw_id.replace('@', '')
            try:
                reply_count = div.find_element_by_xpath('.//div/div[1]/div/div/div[2]/span/span/span').text
            except NoSuchElementException:
                pass
            try:
                retweet_count = div.find_element_by_xpath('.//div/div[2]/div/div/div[2]/span/span/span').text
            except NoSuchElementException:
                pass
            try:
                like_count = div.find_element_by_xpath('.//div/div[3]/div/div/div[2]/span/span/span').text
            except NoSuchElementException:
                pass
        except (NoSuchElementException, StaleElementReferenceException):
            continue
#         print(tw_username)
#         print(tw_id)
#         print(tw_date)
#         print(tw_text)
#         print(reply_count)
#         print(retweet_count)
#         print(like_count)
#         print(tw_url)
#         print('------------------------\n')
        record = (tw_username, tw_id, tw_date, tw_text, reply_count, retweet_count, like_count, tw_url)
        if post_count <= 0:
                break
        elif record:
            print(f'#{post_count} Scrape tweet from {tw_username} post.')
            records.append(record)
            post_count -= 1
    if post_count <= 0:
            break
    y = driver.execute_script("return window.pageYOffset;")
    y += 2000
    driver.execute_script(f'window.scrollTo(0, {y});')
    
driver.close()
print(f"\nScraped. total {len(records)+1} records.")

#50 Scrape tweet from Gmane post.
#49 Scrape tweet from Merkaba X post.
#48 Scrape tweet from Merkaba X post.
#47 Scrape tweet from My Mixtapez post.
#46 Scrape tweet from Shannon Sharpe Burner Account post.
#45 Scrape tweet from Gmane post.
#44 Scrape tweet from Front Office Sports post.
#43 Scrape tweet from Gmane post.
#42 Scrape tweet from Front Office Sports post.
#41 Scrape tweet from Gmane post.
#40 Scrape tweet from Boxing King Media post.
#39 Scrape tweet from FIGHT HUB MARCOS post.
#38 Scrape tweet from Boxing King Media post.
#37 Scrape tweet from FIGHT HUB MARCOS post.
#36 Scrape tweet from Michael Benson post.
#35 Scrape tweet from Boxing undefeated post.
#34 Scrape tweet from DAZN Boxing post.
#33 Scrape tweet from Boxing undefeated post.
#32 Scrape tweet from DAZN Boxing post.
#31 Scrape tweet from TWSN post.
#30 Scrape tweet from SiR ATT9AWT ORAKSALAGH ⵣ post.
#29 Scrape tweet from 2Cool2Blog post.
#28 Scrape tweet from FIGHT HUB MARCOS post.
#27 Scrape tweet from Micha

In [35]:
# show records
df = pd.DataFrame(records)
df = df.drop_duplicates(keep='first')
records = df.values.tolist()
df.columns = col_name

df

,Username,Id,PostDate,Text,Reply,Retweet,Like,TweetURL
0,Gmane,@Gmane07,2022-06-12T23:19:53.000Z,With a perfect record of 50-0 and having total...,383,12.4K,62.9K,https://twitter.com/Gmane07
1,Merkaba X,@_Merkaba_X,2022-06-08T16:36:38.000Z,I wish Floyd Mayweather Sr was healthy enough ...,12,141,679,https://twitter.com/_Merkaba_X
3,My Mixtapez,@mymixtapez,2022-06-14T22:09:23.000Z,NBA Youngboy and Yaya Mayweather’s son is alre...,61,619,"4,939",https://twitter.com/mymixtapez
4,Shannon Sharpe Burner Account,@shannonsharpeee,2022-06-08T01:25:34.000Z,Conor McGregor & Floyd Mayweather Promo was th...,14,202,764,https://twitter.com/shannonsharpeee
5,Gmane,@Gmane07,2022-06-13T02:33:26.000Z,"After being head butted by Victor Ortiz, Floyd...",11,673,"2,481",https://twitter.com/Gmane07
6,Front Office Sports,@FOS,2022-06-15T23:21:00.000Z,Floyd Mayweather has an estimated net worth of...,1,9,13,https://twitter.com/FOS
9,Gmane,@Gmane07,2022-06-13T03:12:57.000Z,"A fight after doing Jail time, Floyd Mayweathe...",25,925,"3,248",https://twitter.com/Gmane07
10,Boxing King Media,@TheBoxingKing1,2022-06-11T09:56:44.000Z,It’s one thing taking advice but actually list...,17,222,"1,415",https://twitter.com/TheBoxingKing1
11,FIGHT HUB MARCOS,@heyitsmarcosv,2022-06-15T23:57:25.000Z,Floyd Sr showing me the ring \n@FloydMayweathe...,10,35,264,https://twitter.com/heyitsmarcosv
14,Michael Benson,@MichaelBensonn,2022-06-14T13:01:22.000Z,Floyd Mayweather with a speech about the numbe...,124,710,"3,277",https://twitter.com/MichaelBensonn


In [36]:
# save data to csv file
search_term = search_term.replace(' ', '_')

from datetime import datetime
cur_date = datetime.today().strftime('%d-%m-%Y')

with open(f'{search_term} {cur_date}.csv', 'w', newline='', encoding='utf-8-sig') as f:
    writer = csv.writer(f)
    writer.writerow(col_name)
    writer.writerows(records)
    
print('finish')

finish
